<a href="https://colab.research.google.com/github/tamaramrt/Mamo.ia/blob/main/Explica%C3%A7%C3%A3o_Extra%C3%A7%C3%A3o_de_atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### 1. **IDEIA GERAL:**

 A ideia geral do codigo é receber como entrada uma imagem e ter como saída o vetor dos atributos extraídos daquela imagem.

 Para isso, ele vai utilizar uma rede neural convolucional, a Lenet (a mesma rede neural utilizada no weka). Essa Lenet será treinada em um conjunto de dados de imagens, chamamos de dataset, que é o CIFAR-100. Ele possui imagens coloridas e 100 categorias diferentes.

 Só explicando melhor, a rede neural convolucional vai processar imagens para aprender padroes e daí extrair os atributos, que serão as "informações relevantes".


### 2. **PASSOS DO CODIGO:**

*   Treinar a Lenet no CIFAR-100
*   Extração de atributos com a Lenet Pré-treinada




### **Para fazer esse processo, é importante entender:**

- Numpy é uma biblioteca muito comum utilizada para manipulação de arrays;

- Pytorch é a biblioteca que vamos usar para a implementação e treinamento da rede neural;

- PIL (PILLOW): é a biblioteca utilizada para a gente carregar as imagens

In [ ]:
!pip install numpy

In [ ]:
!pip install torch

In [ ]:
!pip install torchvision

In [ ]:
!pip install liac-arff

  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=c679f0b7b6ec792ae24fb0319410bd32a79c1767fd0bf55a5e1d1d1d8ef75477
  Stored in directory: /root/.cache/pip/wheels/a9/ac/cf/c2919807a5c623926d217c0a18eb5b457e5c19d242c3b5963a
Successfully built liac-arff


## **1. Treinamento da Lenet no CIFAR-100 (dataset disponivel diretamente pela biblioteca Pythorch)**

**(aqui acredito que para quem ainda não tem muita experiencia com programação, não precisa se preocupar em entender necessariamente linha por linha do codigo, porque tem alguns conceitos mais avançados. Aconselho vcs focarem em entender o geral do que o código vai fazer e como rodar isso)*.
**

Dando uma explicação geral, a LeNet é uma rede neural que aprende a reconhecer padroes em imagens.
Nesse treinamento, ela passa por:

-Camadas convolucionais , são esses conv, conv2 => eles vao identificar as bordas, formas das imagens

-fc1, fc2 são as camadas totalmente conectadas, são elas que vão trasnformar as informações em um vetor de atributos

-RELU é a função de ativação, ela vai ajudar a rede a aprender padrões mais complexos

...

**OBS IMPORTANTE**: Geralmente o treinamento de uma rede neural demora bastante tempo de execução (esse daqui foi mais ou menos 1 hora). Então qnd vcs colocarem para executar e tiver demorando muito, não se preocupem, isso é normal porque a rede vai aprender padrões a partir do CIFAR-100, um conjunto de imagens com 100 classes... Mas cada um precisa colocar para executar porque é a partir desse treinamento que vamos testar a extração de atributos na base de termografia.

**(obs: salvar o arquivo da Lenet pre treinada na maquina para não precisar treinar novamente )**

In [ ]:
import os
os._exit(0)
#codigo para reiniciar o runtime caso esteja dando erro na importação do pytorch


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

#Definicao da arquitetura da Lenet ajustada para CIFAR-100
class LeNetCIFAR(nn.Module):
    def __init__(self):
        super(LeNetCIFAR, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)  # primeira camada convolucional para imagens RGB
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #camada de pooling
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5) #segunda camada convolucional
        self.fc1 = nn.Linear(50 * 5 * 5, 500)  # primeira camada totalmente conectada
        self.fc2 = nn.Linear(500, 100)  # camada de saída para 100 classes (cifar-100)


    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#Definiçao do dispositivo de treinamento
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Transformaçoes para a normalizacao das imagens e conversao para tensores
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#carregar dataset CIFAR-100
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

#criar dataloaders (carregar os dados em lotes)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

#inicializar o modelo, funcao de perda e otimizador
model = LeNetCIFAR().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(num_epochs=10):
  """
  Funcao para treinar o modelo
  """
  for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}")


  torch.save(model.state_dict(), 'lenet_cifar100.pth')
  print("Modelo salvo como 'lenet_cifar100.pth'.")

train_model(num_epochs=10)


***Depois que terminar a execução, já treinamos a Lenet (podem verificar que ela fica salva em um 'lenet_cifar100.pth')!! ***

## 2. Codigo para extração de atributos a partir do modelo pré-treinado

### **Como é feita a extração de atributos de uma imagem:**

*(aqui novamente, tem alguns conceitos mais avançados tbm, então se preocupem mais em entender o geral do que o código vai fazer e como rodar isso -> é importante que todos entendam direitinho como executar)*.



### *(Explicação geral do funcionamento do codigo):*

Primeiro é realizado o pré-processamento da imagem, que basicamente a gente vai ajusatr ela para o formato esperado pela rede (é por isso que tem os passos redimensionamento -> conversao para tensor -> normalização)

Daí passa pela rede Lenet, e a imagem vai percorrer as camadas da rede até a penultima camada fc1 (aqui a gente para em fc1 porque essa camada contém os atributos extraídos da imagem. fc2 faz a classificação final, e isso não é o nosso obejtivo).

A saída é uma sequência de números (tensor), que representa as características extraídas da imagem.

### Aqui são exemplos de 2 imagens onde será feita a extração de atributos pelo arquivo da Lenet que a gente acabou de treinar


**Primeiro, a gente precisa carregar a imagem do drive (no lado esquerdo do colab, tem uma guia, vocês vão em arquivos e fazem o upload da imagem)=> isso todos vão ter que fazer.**

exemplo 1 (/content/Lesao_Benigna_T2_72.jpg)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

# Definição da arquitetura da rede Lenet configurada para extraçao de atributos
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)  #primeira camada convolucional
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #camada de pooling
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5) #segunda camada convolucional
        self.fc1 = nn.Linear(50 * 5 * 5, 500) #penultima camada (onde vai ser feita a extracao de atributos)
        self.fc2 = nn.Linear(500, 100) # camada de saida com 100 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# pre processamento da imagem para torna-la compativel com a Lenet
def preprocessamento_imagem(image_path):

    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # redimensiona imagem para 32x32
        transforms.ToTensor(),  # converte a imagem em tensor
        transforms.Normalize((0.5,0.5, 0.5), (0.5,0.5, 0.5)) #normalizaçao
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0) #
    return image

#Funçao para extrair atributos
def extrair_atributos_img(image_path, model):

    input_image = preprocessamento_imagem(image_path).to(device)
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        # Passar pela conv1, conv2 e obter a penúltima camada
        x = model.pool(torch.relu(model.conv1(input_image)))
        x = model.pool(torch.relu(model.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        features = torch.relu(model.fc1(x))  # extrair os atributos da penúltima camada
    return features

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # instanciar a LeNet e carregar os pesos pre-treinados
    model = LeNet().to(device)
    model.load_state_dict(torch.load('lenet_cifar100.pth'))  # carregar o modelo treinado

    # Caminho para a imagem
    image_path = '/content/Lesao_Benigna_T2_72.jpg'  #copiar o caminho da imagem aqui

    # Extrair os atributos
    features = extrair_atributos_img(image_path, model)
    print(f"Atributos extraídos: {features}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

exemplo 2 (/content/Lesao_Benigna_T2_71.jpg)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

# Definição da arquitetura da rede Lenet configurada para extraçao de atributos
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)  #primeira camada convolucional
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #camada de pooling
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5) #segunda camada convolucional
        self.fc1 = nn.Linear(50 * 5 * 5, 500) #penultima camada (onde vai ser feita a extracao de atributos)
        self.fc2 = nn.Linear(500, 100) # camada de saida com 100 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# pre processamento da imagem para torna-la compativel com a Lenet
def preprocessamento_imagem(image_path):

    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # redimensiona imagem para 32x32
        transforms.ToTensor(),  # converte a imagem em tensor
        transforms.Normalize((0.5,0.5, 0.5), (0.5,0.5, 0.5)) #normalizaçao
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0) #
    return image

#Funçao para extrair atributos
def extrair_atributos_img(image_path, model):

    input_image = preprocessamento_imagem(image_path).to(device)
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        # Passar pela conv1, conv2 e obter a penúltima camada
        x = model.pool(torch.relu(model.conv1(input_image)))
        x = model.pool(torch.relu(model.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        features = torch.relu(model.fc1(x))  # extrair os atributos da penúltima camada
    return features

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # instanciar a LeNet e carregar os pesos pre-treinados
    model = LeNet().to(device)
    model.load_state_dict(torch.load('lenet_cifar100.pth'))  # carregar o modelo treinado

    # Caminho para a imagem
    image_path = '/content/Lesao_Benigna_T2_71.jpg'  #copiar o caminho da imagem aqui

    # Extrair os atributos
    features = extrair_atributos_img(image_path, model)
    print(f"Atributos extraídos: {features}")

### **EXTRAÇÃO DOS ATRIBUTOS DE LESÃO MALIGNA**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import os
import arff
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

image_folder = "/content/drive/MyDrive/LESAO_MALIGNA"


# Definição da arquitetura da rede Lenet configurada para extraçao de atributos
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)  #primeira camada convolucional
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #camada de pooling
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5) #segunda camada convolucional
        self.fc1 = nn.Linear(50 * 5 * 5, 500) #penultima camada (onde vai ser feita a extracao de atributos)
        self.fc2 = nn.Linear(500, 100) # camada de saida com 100 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# pre processamento da imagem para torna-la compativel com a Lenet
def preprocessamento_imagem(image_path):

    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # redimensiona imagem para 32x32
        transforms.ToTensor(),  # converte a imagem em tensor
        transforms.Normalize((0.5,0.5, 0.5), (0.5,0.5, 0.5)) #normalizaçao
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0) #
    return image

#Funçao para extrair atributos
def extrair_atributos_img(image_path, model):

    input_image = preprocessamento_imagem(image_path).to(device)
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        # Passar pela conv1, conv2 e obter a penúltima camada
        x = model.pool(torch.relu(model.conv1(input_image)))
        x = model.pool(torch.relu(model.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        features = torch.relu(model.fc1(x))  # extrair os atributos da penúltima camada
    return features.squeeze().cpu().numpy().tolist()  # Converter para lista e retornar

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # instanciar a LeNet e carregar os pesos pre-treinados
    model = LeNet().to(device)
    model.load_state_dict(torch.load('lenet_cifar100.pth'))  # carregar o modelo treinado

   #criar dataset ARFF
    dataset = {
    'description': 'Atributos extraídos_LESAO MALIGNA',
    'relation': 'Termografia_LESAO_MALIGNA_Atributos',
    'attributes': [('attr' + str(i), 'REAL') for i in range(500)] + [('class', 'STRING')],
    'data': []
    }

# Processar todas as imagens na pasta
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)
        atributos = extrair_atributos_img(image_path, model)
        dataset['data'].append(atributos + [filename])  # Adiciona nome da imagem como rótulo

# Salvar em arquivo ARFF
output_file = "/content/drive/MyDrive/LESAO_MALIGNA/atributos_extracao_LESAO_MALIGNA.arff"
with open(output_file, 'w') as f:
    arff.dump(dataset, f)

print(f"Atributos salvos em {output_file}")

### **EXTRAÇÃO DOS ATRIBUTOS COM TODAS AS CLASSES**

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import os
import arff
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

image_folder = "/content/drive/MyDrive/TERMOGRAFIA_MAMA_IMAGENS"


# Definição da arquitetura da rede Lenet configurada para extraçao de atributos
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)  #primeira camada convolucional
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #camada de pooling
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5) #segunda camada convolucional
        self.fc1 = nn.Linear(50 * 5 * 5, 500) #penultima camada (onde vai ser feita a extracao de atributos)
        self.fc2 = nn.Linear(500, 100) # camada de saida com 100 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# pre processamento da imagem para torna-la compativel com a Lenet
def preprocessamento_imagem(image_path):

    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # redimensiona imagem para 32x32
        transforms.ToTensor(),  # converte a imagem em tensor
        transforms.Normalize((0.5,0.5, 0.5), (0.5,0.5, 0.5)) #normalizaçao
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0) #
    return image

#Funçao para extrair atributos
def extrair_atributos_img(image_path, model):

    input_image = preprocessamento_imagem(image_path).to(device)
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        # Passar pela conv1, conv2 e obter a penúltima camada
        x = model.pool(torch.relu(model.conv1(input_image)))
        x = model.pool(torch.relu(model.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        features = torch.relu(model.fc1(x))  # extrair os atributos da penúltima camada
    return features.squeeze().cpu().numpy().tolist()  # Converter para lista e retornar

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # instanciar a LeNet e carregar os pesos pre-treinados
    model = LeNet().to(device)
    model.load_state_dict(torch.load('lenet_cifar100.pth'))  # carregar o modelo treinado

   #criar dataset ARFF
    dataset = {
    'description': 'Atributos extraídos_TERMOGRAFIA',
    'relation': 'Termografia_Atributos',
    'attributes': [('attr' + str(i), 'REAL') for i in range(500)] + [('class', 'STRING')],
    'data': []
    }

# Processar todas as imagens na pasta
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)
        atributos = extrair_atributos_img(image_path, model)
        dataset['data'].append(atributos + [filename])  # Adiciona nome da imagem como rótulo

# Salvar em arquivo ARFF
output_file = "/content/drive/MyDrive/TERMOGRAFIA_MAMA_IMAGENS/atributos_extracao_TERMOGRAFIA.arff"
with open(output_file, 'w') as f:
    arff.dump(dataset, f)

print(f"Atributos salvos em {output_file}")

Teste com base mama.zip

In [ ]:
# Caminhos utilizados
caminho_zip = 'mama.zip'
destino = 'mama_extraida'

# Função para extrair o arquivo ZIP
def extrair_zip(caminho_zip : str, destino : str):
    """
    Parametros:
        input:
        caminho_zip [str]: Caminho do arquivo ZIP.
        destino [str]: Caminho para a pasta de destino.
    """
    with zipfile.ZipFile(file=caminho_zip, mode='r') as zip_ref:
        zip_ref.extractall(destino)

# Processo de extração e preparação dos dados
extrair_zip(caminho_zip, destino)
imagens, rotulos = carregar_imagens(destino)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import os
import arff
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

image_folder = "/content/drive/MyDrive/TERMOGRAFIA_MAMA_IMAGENS"


# Definição da arquitetura da rede Lenet configurada para extraçao de atributos
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)  #primeira camada convolucional
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #camada de pooling
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5) #segunda camada convolucional
        self.fc1 = nn.Linear(50 * 5 * 5, 500) #penultima camada (onde vai ser feita a extracao de atributos)
        self.fc2 = nn.Linear(500, 100) # camada de saida com 100 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# pre processamento da imagem para torna-la compativel com a Lenet
def preprocessamento_imagem(image_path):

    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # redimensiona imagem para 32x32
        transforms.ToTensor(),  # converte a imagem em tensor
        transforms.Normalize((0.5,0.5, 0.5), (0.5,0.5, 0.5)) #normalizaçao
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0) #
    return image

#Funçao para extrair atributos
def extrair_atributos_img(image_path, model):

    input_image = preprocessamento_imagem(image_path).to(device)
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        # Passar pela conv1, conv2 e obter a penúltima camada
        x = model.pool(torch.relu(model.conv1(input_image)))
        x = model.pool(torch.relu(model.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        features = torch.relu(model.fc1(x))  # extrair os atributos da penúltima camada
    return features.squeeze().cpu().numpy().tolist()  # Converter para lista e retornar

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # instanciar a LeNet e carregar os pesos pre-treinados
    model = LeNet().to(device)
    model.load_state_dict(torch.load('lenet_cifar100.pth'))  # carregar o modelo treinado

   #criar dataset ARFF
    dataset = {
    'description': 'Atributos extraídos_TERMOGRAFIA',
    'relation': 'Termografia_Atributos',
    'attributes': [('attr' + str(i), 'REAL') for i in range(500)] + [('class', 'STRING')],
    'data': []
    }

# Processar todas as imagens na pasta
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)
        atributos = extrair_atributos_img(image_path, model)
        dataset['data'].append(atributos + [filename])  # Adiciona nome da imagem como rótulo

# Salvar em arquivo ARFF
output_file = "/content/drive/MyDrive/TERMOGRAFIA_MAMA_IMAGENS/atributos_extracao_TERMOGRAFIA.arff"
with open(output_file, 'w') as f:
    arff.dump(dataset, f)

print(f"Atributos salvos em {output_file}")

In [ ]:
#Salvar em formato CSV


import torch
import torch.nn as nn
import torchvision.transforms as transforms
import os
import csv
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

image_folder = "/content/drive/MyDrive/TERMOGRAFIA_MAMA_IMAGENS"


# Definição da arquitetura da rede Lenet configurada para extraçao de atributos
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)  #primeira camada convolucional
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #camada de pooling
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5) #segunda camada convolucional
        self.fc1 = nn.Linear(50 * 5 * 5, 500) #penultima camada (onde vai ser feita a extracao de atributos)
        self.fc2 = nn.Linear(500, 100) # camada de saida com 100 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# pre processamento da imagem para torna-la compativel com a Lenet
def preprocessamento_imagem(image_path):

    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # redimensiona imagem para 32x32
        transforms.ToTensor(),  # converte a imagem em tensor
        transforms.Normalize((0.5,0.5, 0.5), (0.5,0.5, 0.5)) #normalizaçao
    ])

    image = Image.open(image_path)
    image = transform(image).unsqueeze(0) #
    return image

#Funçao para extrair atributos
def extrair_atributos_img(image_path, model):

    input_image = preprocessamento_imagem(image_path).to(device)
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():
        # Passar pela conv1, conv2 e obter a penúltima camada
        x = model.pool(torch.relu(model.conv1(input_image)))
        x = model.pool(torch.relu(model.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        features = torch.relu(model.fc1(x))  # extrair os atributos da penúltima camada
    return features.squeeze().cpu().numpy().tolist()  # Converter para lista e retornar

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # instanciar a LeNet e carregar os pesos pre-treinados
    model = LeNet().to(device)
    model.load_state_dict(torch.load('/content/lenet_cifar100.pth'))  # carregar o modelo treinado

   #caminho do csv de saida
    output_csv = "/content/drive/MyDrive/TERMOGRAFIA_MAMA_IMAGENS/atributos_extracao_TERMOGRAFIA.csv"

   #cabeçalhos
    headers = ['attr' + str(i) for i in range(500)] + ['class']


    # Escreve os dados
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)

        for filename in os.listdir(image_folder):
            if filename.lower().endswith(('.jpg', '.png')):
                image_path = os.path.join(image_folder, filename)
                atributos = extrair_atributos_img(image_path, model)
                writer.writerow(atributos + [filename])

    print(f"Atributos salvos com sucesso em: {output_csv}")

